# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://google.com")
ed.links

['https://mail.google.com/mail/&ogbl',
 'https://www.google.com/imghp?hl=en&ogbl',
 'https://www.google.com.my/intl/en/about/products',
 'https://accounts.google.com/ServiceLogin?hl=en&passive=true&continue=https://www.google.com/&ec=futura_exp_og_so_72776762_e',
 'https://www.google.com/setprefs?sig=0_CZwiTLclF8Rsh0bkUPQhH-K-4kg%3D&hl=ms&source=homepage&sa=X&ved=0ahUKEwiYpcH29YiPAxXc6qACHTpiEwkQ2ZgBCBU',
 'https://about.google/?utm_source=google-MY&utm_medium=referral&utm_campaign=hp-footer&fg=1',
 'https://www.google.com/intl/en_my/ads/?subid=ww-ww-et-g-awa-a-g_hpafoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpafooter&fg=1',
 'https://www.google.com/services/?subid=ww-ww-et-g-awa-a-g_hpbfoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpbfooter&fg=1',
 'https://google.com/search/howsearchworks/?fg=1',
 'https://policies.google.com/privacy?hl=en-MY&fg=1',
 'https://policies.google.com/terms?hl=en-MY&fg=1',
 'https://www.google.com/pr

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://google.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://mail.google.com/mail/&ogbl
https://www.google.com/imghp?hl=en&ogbl
https://www.google.com.my/intl/en/about/products
https://accounts.google.com/ServiceLogin?hl=en&passive=true&continue=https://www.google.com/&ec=futura_exp_og_so_72776762_e
https://www.google.com/setprefs?sig=0_CZwiTLclF8Rsh0bkUPQhH-K-4kg%3D&hl=ms&source=homepage&sa=X&ved=0ahUKEwiYpcH29YiPAxXc6qACHTpiEwkQ2ZgBCBU
https://about.google/?utm_source=google-MY&utm_medium=referral&utm_campaign=hp-footer&fg=1
https://www.google.com/intl/en_my/ads/?subid=ww-ww-et-g-awa-a-g_hpafoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpafooter&fg=1
https://www.google.com/services/?subid=ww-ww-et-g-awa-a-g_hpbfoot1_1!o2&u

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/openai/gpt-oss-120b',
 '/openai/gpt-oss-20b',
 '/Qwen/Qwen-Image',
 '/zai-org/GLM-4.5V',
 '/rednote-hilab/dots.ocr',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Qwen/Qwen-Image',
 '/spaces/amd/gpt-oss-120b-chatbot',
 '/spaces/black-forest-labs/FLUX.1-Krea-dev',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/jxm/gpt-oss20b-samples',
 '/datasets/nvidia/Llama-Nemotron-VLM-Dataset-v1',
 '/datasets/allenai/WildChat-4.8M',
 '/datasets/openai/BrowseCompLongContext',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/d

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
openai/gpt-oss-120b
Updated
about 19 hours ago
•
490k
•
3.3k
openai/gpt-oss-20b
Updated
about 19 hours ago
•
2.37M
•
2.88k
Qwen/Qwen-Image
Updated
8 days ago
•
69.7k
•
1.55k
zai-org/GLM-4.5V
Updated
2 days ago
•
622
•
370
rednote-hilab/dots.ocr
Updated
1 day ago
•
17.9k
•
660
Browse 1M+ models
Spaces

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nopenai/gpt-oss-120b\nUpdated\nabout 19 hours ago\n•\n490k\n•\n3.3k\nopenai/gpt-oss-20b\nUpdated\nabout 19 hours ago\n•\n2.37M\n•\n2.88k\nQwen/Qwen-Image\nUpdated\n8 days ago\n•\n69.7k\n•\n1.55k\nzai-org/GLM-4.5V\nUpdated\n2 days ago\n•\n622\n•\n370\nrednote-hilab/dots.ocr\nUpdated\n1 day ago\n•\n17.9k\n•\n660\nBrowse 1M+ models\nSpaces\nRunning\n11.8k\n11.8k\nDeepSite v2\n🐳\nGenerate any a

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [19]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Brochure

## Welcome to Hugging Face!

**Hugging Face** is the leading platform for the machine learning community, where individuals and organizations collaborate on models, datasets, and applications to shape the future of Artificial Intelligence. We are dedicated to building innovative solutions that empower developers and researchers alike.

### What We Offer

- **Models:** Explore over **1 million models**, including cutting-edge options like `openai/gpt-oss-120b`, and many more that are trending every week.  
- **Datasets:** Access a rich repository of **250,000+ datasets** for any machine learning tasks to enhance your projects.
- **Spaces:** Create and deploy AI applications seamlessly with **400,000+ applications** available for exploration.

### Our Core Values

At Hugging Face, we embrace a vibrant and collaborative company culture that thrives on community involvement. Here's what you can expect:

- **Open-Source Mindset:** We believe in building the foundation of machine learning tools with contributions from the community.
- **Innovation:** We encourage creative problem-solving through our frameworks, including **Transformers**, **Diffusers**, **Tokenizers**, and more.
- **Diversity & Inclusion:** We are committed to building a diverse and inclusive workforce where everyone's voice matters.

### Our Customers

Hugging Face serves over **50,000 organizations**, including industry giants like:

- **Meta**
- **Amazon**
- **Google**
- **Microsoft**

These organizations leverage our platform to streamline their AI development processes and access state-of-the-art machine learning resources.

### Career Opportunities

Join our dynamic team at Hugging Face! We are looking for passionate individuals who want to contribute to the AI community and help shape the future of machine learning. 

**We are hiring across various roles:**
- Software Engineers
- Data Scientists
- Community Managers
- Product Managers

Be part of a culture that values innovation, collaboration, and growth! 

### Connect with Us

- **Website:** [Hugging Face](https://huggingface.co)
- **Social Media:** 
  - [Twitter](https://twitter.com/huggingface)
  - [LinkedIn](https://www.linkedin.com/company/huggingface)
  - [Discord](https://discord.gg/huggingface)

---

Explore, innovate and collaborate with Hugging Face – the future of AI awaits you!

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Welcome to Hugging Face

### The AI Community Building the Future
At Hugging Face, we are more than a company; we are a thriving community dedicated to advancing machine learning technology. We provide a collaborative platform where the machine learning community can explore, develop, and deploy cutting-edge models, datasets, and applications.

---

## Our Offerings

- **Models:** Explore over 1 million state-of-the-art models tailored for a variety of tasks including text, image, audio, video, and 3D.
- **Datasets:** Access and share over 250,000 datasets, providing the resources you need for effective machine learning efforts.
- **Spaces:** Collaborate on running applications; currently hosting over 11,800 innovative AI applications.
- **Enterprise Solutions:** Our enterprise packages offer comprehensive solutions that include dedicated support, enhanced security, and performance optimizations tailored for businesses.

---

## Customer Base

Join a robust community of over 50,000 organizations, including industry giants like:
- **Meta**
- **Google**
- **Amazon**
- **Microsoft**

Our customers span sectors from non-profits to leading tech companies, all utilizing our tools to elevate their machine learning capabilities.

---

## Company Culture

At Hugging Face, we believe in open source and community collaboration as the backbone of innovation. Our culture is rooted in diversity, inclusivity, and shared learning, creating an enriching environment for all team members. We encourage curiosity and creativity, valuing unique perspectives while promoting collaborative problem-solving methodologies.

---

## Careers at Hugging Face

We are always looking for passionate individuals to join our team. If you are excited about machine learning, AI innovation, and being part of a mission-driven community, we want to hear from you!

### Why Work With Us?
- **Impact:** Contribute to revolutionary technology trusted by thousands of organizations.
- **Growth:** Work with and learn from leading experts in the field.
- **Community:** Become part of a collaborative and supportive team.

Visit our [Careers Page](#) to explore current job openings and learn more about the opportunities at Hugging Face.

---

## Get Started

Ready to embark on your machine learning journey with Hugging Face? 
Explore our platform today:

- **[Browse Models](#)**
- **[Access Datasets](#)**
- **[Join our Community](#)**

---

**Contact Us:** For inquiries or additional information, please reach out via our official channels on [Twitter](#), [LinkedIn](#), or [Discord](#). 

### Hugging Face - Together, we're building the future of AI!